# Web Scraping con Python
Vamos a usar las librerías [Requests](https://requests.kennethreitz.org/en/master/) y [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) para hacer web scraping. Instalación (en Anaconda):
```
conda install requests
conda install beautifulsoup4
```

In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
r = requests.get("https://idal.uv.es/ejemplo.html")

In [5]:
type(r)

requests.models.Response

El objeto devuelto es un objeto `requests.models.Response` con unos atributos determinados:

In [6]:
[s for s in dir(r) if not s.startswith('_')]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Podemos ver el código de respuesta del servidor (útil para detectar errores 4XX o 5XX):

In [7]:
r.status_code

200

La URL de donde ha sacado los datos es:

In [8]:
r.url

'https://idal.uv.es/ejemplo.html'

Para obtener la respuesta completa (código HTML de la página) como una cadena de texto accedemos al atributo `text`:

In [9]:
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>Página de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>Página de ejemplo</h1>
    <p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>

Para comprobar el tipo de respuesta (Content Type) y ver si es HTML, JSON, XML, etc.:

In [10]:
r.headers["content-type"]

'text/html; charset=UTF-8'

Si el servidor diera la información de la codificación incorrecta (p. ej. ISO-8859-1 para una página UTF-8) podemos forzar una nueva codificación

In [12]:
r.encoding

'UTF-8'

In [13]:
r.encoding='ISO-8859-1' #cambiamos a ISO-8859-1 para comprobar que muestra mal los acentos
print(r.text)

<!DOCTYPE html>
<html lang="es">
<head>
  <title>PÃ¡gina de ejemplo</title>
  <meta charset="UTF-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
</head>
<body>

  <div class="jumbotron text-center">
    <h1>PÃ¡gina de ejemplo</h1>
    <p>PÃ¡gina <em>HTML</em> con texto de ejemplo para hacer web scraping</p> 
  </div>
  
  <div class="container">
    <div class="row">
      <div class="col-sm-4">
        <h3>Columna 1</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...</p>
        <a class="btn" href="#" title="IDAL">Enlace 1</a>
      </div>
      <div class="col-sm-4">
        <h3>Columna 2</h3>
        <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit...</p>
        <p>Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris...<

Volvemos a la codificación correcta

In [14]:
r.encoding='UTF-8'

Podemos ver todas las cabeceras devueltas:

In [15]:
r.headers

{'Date': 'Wed, 17 Apr 2024 07:42:51 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Last-Modified': 'Wed, 10 Feb 2021 10:11:14 GMT', 'ETag': '"73d-5baf89f8467b1-gzip"', 'Accept-Ranges': 'bytes', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '673', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=UTF-8'}

O una cabecera en concreto

In [16]:
r.headers['Content-Type']

'text/html; charset=UTF-8'

### Extraer contenido del HTML
Usar expresiones regulares para buscar patrones HTML no está recomendado. Es mejor usar una librería para "parsear" el HTML y extraer el contenido de las etiquetas específicas.  
Aún así, las RegEx se pueden usar para buscar patrones específicos como listas de precios, direcciones de correo, números de teléfono.  
Podemos ejecutar una RegEx en el texto de respuesta para buscar un patrón específico, como p. ej. una URL:

In [17]:
import re

re.findall(r'https?://[\w_./]+', r.text)

['https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css',
 'https://idal.uv.es',
 'http://etse.uv.es',
 'http://uv.es']

In [ ]:
patron = re.compile(r'https?://[\w_./]+')

re.search(patron, r.text)

## Librería BeautifulSoup
La librería BeautifulSoup se utiliza para extraer contenidos desde una página HTML. Tiene una API muy simple a la vez que potente.  
Para empezar, hay que convertir el HTML del texto de respuesta en una estructura DOM jerárquica que se pueda recorrer y buscar.


In [18]:
soup = BeautifulSoup(r.text, "html.parser")

El objeto `soup` contiene el DOM de todo el documento HTML

In [19]:
type(soup)

bs4.BeautifulSoup

Podemos referirnos a cada elemento del DOM por su nombre (etiqueta HTML) dentro del objeto `soup` (si hay varias apariciones sólo devuelve la primera)

In [20]:
soup.title

<title>Página de ejemplo</title>

In [21]:
type(soup.title)

bs4.element.Tag

In [22]:
titulo = soup.title.text
print(titulo)

Página de ejemplo


In [23]:
soup.a

<a class="btn" href="#" title="IDAL">Enlace 1</a>

Atributos del elemento HTML

In [24]:
soup.a['href']

'#'

Si un elemento tiene otros elementos anidados los muestra todos.

In [25]:
soup.div

<div class="jumbotron text-center">
<h1>Página de ejemplo</h1>
<p>Página <em>HTML</em> con texto de ejemplo para hacer web scraping</p>
</div>

El atributo `text` contiene el texto de todos los elementos internos.

In [26]:
soup.div.text

'\nPágina de ejemplo\nPágina HTML con texto de ejemplo para hacer web scraping\n'

In [ ]:
soup.div.h1.text

El método `find` busca la primera aparición de una etiqueta en el objeto `Soup`. Cada etiqueta contiene todo su contenido dentro de la estructura DOM.

In [ ]:
link = soup.find("a")
link

Es equivalente a:

In [ ]:
soup.a

In [ ]:
link.text

De cada elemento podemos obtener sus atributos

In [ ]:
link.get('class') #equivalente a link['class']

In [ ]:
link.attrs

In [ ]:
link.attrs['title']

In [ ]:
link['href']

Podemos encadenar varias etiquetas a buscar

In [ ]:
soup.find('p').find('em')

In [ ]:
soup.p.em

Podemos acotar la búsqueda a una clase específica de la etiqueta (p.ej. `<a class="enlace">...</a>`)

In [ ]:
tag = soup.find("a", class_="enlace")
print(tag)

In [ ]:
#o simplemente
tag = soup.find("a", "enlace")
print(tag)

Podemos buscar una etiqueta con un atributo ID específico (p.ej.: `<div id="col4">...</div>`)

In [ ]:
tag = soup.find("div", id="col4")
print(tag)

Podemos buscar un atributo sin especificar el tipo de elemento que lo contiene:

In [ ]:
soup.find(id="enlaces")

In [ ]:
soup.find(href="#")

In [ ]:
soup.find(class_="col-sm-4").h3.text

In [ ]:
soup.find(class_="col-sm-4")

In [ ]:
print(soup.find(class_="col-sm-4").text)

EL objeto que devuelve `find()` es del tipo `Tag`


In [ ]:
type(tag)

In [ ]:
#atributos y métodos de Tag
[s for s in dir(tag) if not s.startswith('_')]

El método `find_all` devuelve una lista de objetos `Tag` con todas las etiquetas que encuentra con ese parámetro de búsqueda.  
P. ej. para buscar todas las etiquetas de un tipo en el árbol DOM (por ejemplo los enlaces `<a>` de una página):

In [ ]:
soup.find_all("a")

Devuelve un elemento especial que funciona como un *iterable*

In [ ]:
type(soup.find_all("a"))

In [ ]:
type(soup.find_all("a")[0])

In [ ]:
#hay un shortcut para find_all
soup("a")

In [ ]:
#No confundir con
soup.a

Que equivale a

In [ ]:
soup("a")[0]

Podemos especificar una lista de etiquetas a buscar y las devuelve todas en el orden en que aparecen en la página.

In [ ]:
soup.find_all(["a", "p"])

También podemos especificar el criterio de búsqueda de las etiquetas mediante expresiones regulares.

In [ ]:
soup.find_all(re.compile("h\d"))

Podemos filtrar por un atributo del `Tag` (por defecto es la clase si no indicamos otra cosa)

In [ ]:
for link in soup.find_all('a', class_="enlace"):
    print(link)

Cada elemento de la lista devuelta es del tipo `Tag` con sus atributos particulares.

In [ ]:
for link in soup.find_all('a', class_="enlace"):
    print(link["href"])

In [ ]:
enlaces = soup.find_all("a", "btn") #equivale a soup.find_all("a", class_="btn")
for e in enlaces:
    print(e)

Podemos hacer una búsqueda de otra etiqueta anidada dentro de la primera búsqueda (útil por ejemplo para buscar elementos genéricos dentro de una sección específica de la página)

In [ ]:
tags = soup.find("div", id="col4").find_all("a")
print(tags)

Repetimos esta búsqueda especificando un selector CSS utilizando el método `select` (más simple que lo anterior si se conoce la sintaxis CSS). Este método devuelve una lista con todos los elementos encontrados.    

In [ ]:
#dado que
soup.select("#col4") #equivale a soup.find_all(id="col4")

In [ ]:
soup.select("a") #equivale a soup.find_all("a")

P. ej. elementos `p` contenidos dentro de un elemento con ID="col4"

In [ ]:
soup.select("#col4 p")

El atributo `contents` del resultado de una etiqueta individual (objeto `bs4.element.Tag`) contiene una lista de objetos con su contenido interno (que incluye tanto los nodos de texto como la representación e texto de todo el HTML anidado).  
Si el objeto sólo contiene texto es del tipo `NavigableString`, y si contiene etiqueta es del tipo `Tag`. Los saltos de línea (`\n`) entre las etiquetas se consideran elementos de texto y aparecen en la lista devuelta.

In [ ]:
print(soup.find("title"))

In [ ]:
inner_contents = soup.find("title").contents
print(inner_contents)

In [ ]:
type(inner_contents)

In [ ]:
soup.find("div", id="col4")

In [ ]:
inner_contents = soup.find("div", id="col4").contents
print(inner_contents)

In [ ]:
inner_contents[0]

In [ ]:
type(inner_contents[0])

In [ ]:
inner_contents[1]

In [ ]:
type(inner_contents[1])

El atributo `text` del objeto `bs4.element.Tag` contiene un string con los textos contenidos en la etiqueta, ignorando todas las etiquetas HTML (como por ejemplo `<span>`, `<strong>` o `<i>`):

In [ ]:
inner_text = soup.find("div", id="col4").text
inner_text

In [ ]:
type(inner_text)

In [ ]:
print(inner_text.strip())

Aquí el método `strip()` simplemente elimina las líneas en blanco (`\n`) del resultado.  

### Búsqueda por función
Podemos definir una función booleana para hacer la búsqueda de elementos en el árbol DOM

In [ ]:
def etiquetas(tag):
    return tag.has_attr('id')

soup.find_all(etiquetas)

In [ ]:
[(t.name, t.attrs) for t in soup.find_all(etiquetas)]

## Navegación por la estructura DOM
Cada elemento `Tag` guarda información de su posición dentro del árbol de etiquetas HTML del documento (estructura DOM) de manera que se puede navegar desde cada etiqueta a sus etiquetas relacionadas:
### Navegar hacia abajo
Podemos usar los nombres de las etiquetas como atributos encadenados del objeto `soup` para recorrer el árbol hacia abajo:

In [ ]:
soup.div.h1 #busca la primera etiqueta <h1> dentro de la primera etiqueta <div> 

In [ ]:
soup("div")[1].h3 #busca la primera etiqueta h3 dentro del elemento 1 de la lista de etiquetas div

Todos los hijos de una etiqueta están en su atributo `contents`:

In [ ]:
soup.div.contents

In [ ]:
type(soup.div.contents)

Los hijos de una etiqueta que sólo contienen texto aparecen en la lista de `contents` como un objeto `NavigableString`. Los que contienen una etiqueta son objetos `Tag`

In [ ]:
for t in soup.div.contents:
    print(type(t))

In [ ]:
soup.div.h1.contents

In [ ]:
type(soup.div.h1.contents[0])

Podemos iterar sobre los descendientes de una etiqueta con el iterador `children`:

In [ ]:
soup.div.children

In [ ]:
[child for child in soup.div.children]

In [ ]:
#si convertimos el iterador de children a lista es equivalente al método contents
list(soup.div.children)==soup.div.contents

Tanto `contents` como `children` consideran sólo los descencientes directos de una etiqueta. Con `descendants` accedemos iterativamente a todos sus descendientes:

In [ ]:
soup.div.p

In [ ]:
soup.div.p.contents

In [ ]:
soup.div.p.descendants

In [ ]:
[child for child in soup.div.p.descendants]

In [ ]:
[child for child in soup.find(id="col4").descendants]

Para obtener sólo el texto de una etiqueta usamos `string` o para obtener iterativamente todos los textos contenidos usamos el iterador `strings`:

In [ ]:
soup.div.h1.string

In [ ]:
#si una etiqueta contiene otras etiquetas el atributo string está vacío
soup.div.p

In [ ]:
soup.div.p.string

In [ ]:
#iteramos por todos los elementos contenidos para obtener su string
[s for s in soup.div.p.strings]

### Navegar hacia arriba
Con `parent` obtenemos la etiqueta superior a una dada, y con `parents` obtenemos iterativamente todos sus ascendientes:

In [ ]:
soup.div.h1.parent

In [ ]:
print(soup.find("div", id="col4").parent)

In [ ]:
for p in soup.find("div", id="col4").parents:
    print(p.name, p.attrs)

In [ ]:
for p in soup.find("h1").parents:
    print(p.name, p.attrs)

### Navegar hacia los lados
Los métodos `next_sibling` y `previous_sibling` acceden a la etiqueta posterior o anterior del mismo nivel. Los métodos `next_siblings` y `previous_siblings` lo hacen iterativamente con todos los elementos al mismo nivel.

In [ ]:
soup.find("div", id="enlaces")

In [ ]:
soup.find("div", id="enlaces").ul.contents

In [ ]:
soup.find("div", id="enlaces").li

In [ ]:
soup.find("div", id="enlaces").li.previous_sibling

In [ ]:
soup.find("div", id="enlaces").li.next_sibling.next_sibling

In [ ]:
#buscamos elementos hermanos a continuación del <li> intermedio
enlace_etse = soup.find("div", id="enlaces").find("a",{"title": "ETSE"})
enlace_etse

In [ ]:
[tag for tag in enlace_etse.parent.next_siblings]

In [ ]:
#buscamos elementos hermanos anteriores al último <li>

[tag for tag in soup.find("div", id="enlaces").find("a",{"title": "UV"}).parent.previous_siblings]

### Ejemplo completo
Del siguiente código HTML vamos a extraer los ítems listados y su precio

In [ ]:
texto = """
<body>
<div id="listings_prices">
 <div class="item">
  <li class="item_name">Watch</li>
  <div class="main_price">Price: $66.68</div>
  <div class="discounted_price">Discounted price: $46.68</div>
   </div>
   <div class="item">
  <li class="item_name">Watch2</li>
  <div class="main_price">Price: $56.68</div>
   </div>
</div>
</body>
"""

In [ ]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    print(product.find("li").text)

In [ ]:
soup = BeautifulSoup(texto, "html.parser")
for product in soup.find_all("div", "item"):
    product_title = product.find("li").text 
    product_price = re.search(r'\$\d+\.\d+', product.find("div", "main_price").text)[0]
    print(f"{product_title} is selling for {product_price}")